In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator
import csv
from SIR import SIR_network

# SIR参数设置
beta = 0.1  # 感染率
gamma = 0.5  # 免疫率
step = 20  # 迭代次数

markers = ['<', '>', '^', 'v', 'o', 's', 'D', '*', 'x', '+']  # plt marker大全


# 获得所需的top-k节点
def getTopK(methods, k):
    print('Top-' + str(k) + '节点生成中...')
    df = pd.DataFrame(columns=methods_name)  # 存储各个方法的top-k节点
    # 循环每一个方法
    for i in range(len(methods_name)):
        method = methods.iloc[:, i + 1]  # 方法列
        top_K = method.argsort()[::-1][0:k]  # top-k,此处为索引号，与Id号差1
        df[methods_name[i]] = top_K  # 存储到dataframe
    return df


if __name__ == '__main__':
    node_path = r'C:\Users\zzsgdsg7\Desktop\netnode18.csv'
    edge_path = r'C:\Users\zzsgdsg7\Desktop\netedge18.csv'

    with open(node_path) as nodecsv:
        nodereader = csv.reader(nodecsv)
        nodes = [n for n in nodereader][1:]
    node_names = [n[0] for n in nodes]

    with open(edge_path) as edgecsv:  # Open the file
        edgereader = csv.reader(edgecsv)  # Read the csv
        edges = [tuple(e) for e in edgereader][1:]  # Retrieve the data

    G = nx.Graph()
    G.add_nodes_from(node_names)
    G.add_edges_from(edges)

    dataset = 'dataset'  # 数据集/网络名称
    # 数据
    #adj = np.loadtxt('adj.csv', dtype=np.int)  # 邻接矩阵
    file_path = r'C:\Users\zzsgdsg7\Desktop\多种排序方式.csv'
    metheds = pd.read_csv(file_path) # 多种方法的排序结果

    print(metheds)
    #graph = nx.from_numpy_matrix(adj)  # 网络图
    graph = G
    # 读取方法文件的列名
    methods_name = np.array(metheds.columns)[1: len(metheds.columns)]
    print('文件中共有以下 ' + str(len(methods_name)) + ' 种方法')
    print(methods_name)

    # 选择需要画图显示的方法，可自行调整，也可设置为手动输入
    plt_methods = [1, 4, 5]

    # 可自行设置Top-K的具体k值
    for k in range(5, 20, 5):
        print('Top-%d...' % k)
        df = getTopK(metheds, k)  # 多种方法的Top-k结果

        sir_values_list = []  # 存储在该Top-k下各方法的sir感染情况
        # 循环所有方法
        for name in methods_name:
            sir_source = np.array(df[name])  # df转为数组
            sir_values = SIR_network(graph, sir_source, beta, gamma, step)  # sir传播
            sir_values_list.append(sir_values)  # 存储每个方法的Sir传播情况

        # 输出F(tc)
        for i in plt_methods:
            print(methods_name[i], sir_values_list[i][step])

        # SIR传播曲线结果可视化
        fig = plt.figure(figsize=(10, 6), dpi=150)  # 创建画布
        plt.title(dataset + ' Top-' + str(k))  # 设置标题
        plt.xticks(fontproperties='Times New Roman', size=16)  # 设置字体
        plt.yticks(fontproperties='Times New Roman', size=16)  # 设置字体
        plt.xlabel('t', fontproperties='Times New Roman', size=16)  # x轴标题
        plt.ylabel('F(t)', fontproperties='Times New Roman', size=16)  # y轴标题:第t个时间迭代步时的感染态和免疫态的数量之和F(t)
        ax = plt.gca()  # ax为两条坐标轴的实例
        ax.xaxis.set_major_locator(MultipleLocator(2))  # 把x轴的刻度间隔设置为2，并存在变量里

        plt_labels = []  # 存储需要plt的方法名，在图例中进行设置
        # 画图
        for i in plt_methods:
            plt.plot(range(step + 1), sir_values_list[i], marker=markers[i], linewidth=1.5, ms=10)
            plt_labels.append(methods_name[i])

        # 添加图例
        plt.legend(labels=plt_labels,
                   prop={'family': 'Times New Roman',
                         'weight': 'normal',
                         'size': 16},
                   loc="lower right")
        plt.show()

ModuleNotFoundError: No module named 'SIR'